# Tutorial

Intake-thredds provides an interface that combines functionality from [`siphon`](https://github.com/Unidata/siphon) and `intake` to retrieve data from THREDDS data servers. This tutorial provides an introduction to the API and features of intake-thredds. Let's begin by importing `intake`. 

In [1]:
import intake

## Loading a catalog

You can load data from a THREDDS catalog by providing the URL to a valid THREDDS catalog: 

In [2]:
cat_url = 'https://psl.noaa.gov/thredds/catalog/Datasets/noaa.ersst/catalog.xml'

In [3]:
catalog = intake.open_thredds_cat(cat_url, name='noaa-ersst-catalog')
print(catalog)
print(type(catalog))

<Intake catalog: noaa-ersst-catalog>
<class 'intake_thredds.cat.ThreddsCatalog'>


## Using the catalog

Once you've loaded a catalog, you can display its contents by iterating over its entries:

In [4]:
list(catalog)

['err.mnmean.v3.nc',
 'sst.mnmean.v3.nc',
 'sst.mnmean.v4.nc',
 'sst.mon.1971-2000.ltm.v4.nc',
 'sst.mon.19712000.ltm.v3.nc',
 'sst.mon.1981-2010.ltm.v3.nc',
 'sst.mon.1981-2010.ltm.v4.nc']

Once you've identified a dataset of interest, you can access it as follows:

In [5]:
source = catalog['err.mnmean.v3.nc']
print(source)

sources:
  err.mnmean.v3.nc:
    args:
      chunks: {}
      urlpath: https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst/err.mnmean.v3.nc
    description: THREDDS data
    driver: intake_xarray.opendap.OpenDapSource
    metadata:
      catalog_dir: null



In [6]:
print(type(source))

<class 'intake_xarray.opendap.OpenDapSource'>


## Loading a dataset

To load a dataset of interest, you can use the `to_dask()` method which is available on a **source** object:

In [7]:
%%time
ds = source().to_dask()
ds

CPU times: user 248 ms, sys: 24.2 ms, total: 273 ms
Wall time: 3.92 s


<xarray.Dataset>
Dimensions:    (lat: 89, lon: 180, nbnds: 2, time: 1994)
Coordinates:
  * lat        (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon        (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time       (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2020-02-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(1994, 2), meta=np.ndarray>
    err        (time, lat, lon) float32 dask.array<chunksize=(1994, 89, 180), meta=np.ndarray>
Attributes:
    title:                           NOAA Extended Reconstructed SST V3
    Conventions:                     CF-1.0
    history:                         Thu Jul  1 14:04:15 2010: ncatted -O -a ...
    comments:                        The extended reconstructed sea surface t...
    platform:                        Model
    source:                          NOAA/NESDIS/National Climatic Data Center
    institution:                     NOAA/NESDIS/National Climatic Data Center
    citation:                        Smith, T.M., R.W. Reynolds, Thomas C. Pe...
    dataset_title:                   Extended Reconstructed Sea Surface Tempe...
    source_doc:                      https://www.ncdc.noaa.gov/data-access/ma...
    References:                      https://www.psl.noaa.gov/data/gridded/da...
    DODS_EXTRA.Unlimited_Dimension:  time

The `to_dask()` reads only metadata needed to construct an ``xarray.Dataset``. The actual data are streamed over the network when computation routines are invoked on the dataset. 
By default, `intake-thredds` uses ``chunks={}`` to load the dataset with dask using a single chunk for all arrays. You can use a different chunking scheme by prividing a custom value of chunks before calling `.to_dask()`:

In [8]:
%%time
# Use a custom chunking scheme
ds = source(chunks={'time': 100, 'lon': 90}).to_dask()
ds

CPU times: user 201 ms, sys: 19.9 ms, total: 221 ms
Wall time: 3.93 s


<xarray.Dataset>
Dimensions:    (lat: 89, lon: 180, nbnds: 2, time: 1994)
Coordinates:
  * lat        (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon        (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time       (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2020-02-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(100, 2), meta=np.ndarray>
    err        (time, lat, lon) float32 dask.array<chunksize=(100, 89, 90), meta=np.ndarray>
Attributes:
    title:                           NOAA Extended Reconstructed SST V3
    Conventions:                     CF-1.0
    history:                         Thu Jul  1 14:04:15 2010: ncatted -O -a ...
    comments:                        The extended reconstructed sea surface t...
    platform:                        Model
    source:                          NOAA/NESDIS/National Climatic Data Center
    institution:                     NOAA/NESDIS/National Climatic Data Center
    citation:                        Smith, T.M., R.W. Reynolds, Thomas C. Pe...
    dataset_title:                   Extended Reconstructed Sea Surface Tempe...
    source_doc:                      https://www.ncdc.noaa.gov/data-access/ma...
    References:                      https://www.psl.noaa.gov/data/gridded/da...
    DODS_EXTRA.Unlimited_Dimension:  time

## Working with nested catalogs

In some scenarious, a THREDDS catalog can reference another THREDDS catalog. This results into a nested structure consisting of a parent catalog and children catalogs:

In [9]:
cat_url = 'https://psl.noaa.gov/thredds/catalog.xml'
catalog = intake.open_thredds_cat(cat_url)
list(catalog)

['Datasets', 'Projects', 'Aggregations']

In [10]:
print(list(catalog['Datasets']))

['20thC_ReanV2c', '20thC_ReanV3', 'ATOMIC', 'COBE', 'COBE2', 'CarbonTracker', 'E3SM', 'E3SM_LE', 'LIM', 'NARR', 'S2S', 'SERDP_regimeshifts', 'Timeseries', 'cmap', 'coads', 'cpc_global_precip', 'cpc_global_temp', 'cpc_us_hour_precip', 'cpc_us_precip', 'cpcsoil', 'cru', 'dai_pdsi', 'ghcncams', 'ghcngridded', 'gistemp', 'godas', 'gpcc', 'gpcp', 'icoads', 'icoads2.5', 'interp_OLR', 'jmatemp', 'kaplan_sst', 'livneh', 'mlost', 'mlostv3b', 'ncep', 'ncep.marine', 'ncep.pac.ocean', 'ncep.reanalysis', 'ncep.reanalysis.dailyavgs', 'ncep.reanalysis.derived', 'ncep.reanalysis2', 'ncep.reanalysis2.dailyavgs', 'ncep.reanalysis2.derived', 'noaa.ersst', 'noaa.ersst.v3', 'noaa.ersst.v4', 'noaa.ersst.v5', 'noaa.oisst.v2', 'noaa.oisst.v2.derived', 'noaa.oisst.v2.highres', 'noaa_hrc', 'noaaglobaltemp', 'noaamergedtemp', 'nodc.woa94', 'nodc.woa98', 'olrcdr', 'prec', 'precl', 'rte-rrtmgp', 'snowcover', 'udel.airt.precip', 'uninterp_OLR']


In [11]:
print(list(catalog['Datasets']['ncep.reanalysis.dailyavgs']))

['other_gauss', 'pressure', 'surface', 'surface_gauss', 'tropopause']


In [12]:
print(list(catalog['Datasets']['ncep.reanalysis.dailyavgs']['surface'])[:10])

['air.sig995.1948.nc', 'air.sig995.1949.nc', 'air.sig995.1950.nc', 'air.sig995.1951.nc', 'air.sig995.1952.nc', 'air.sig995.1953.nc', 'air.sig995.1954.nc', 'air.sig995.1955.nc', 'air.sig995.1956.nc', 'air.sig995.1957.nc']


To load data from such a nested catalog, `intake-thredds` provides a special source object {py:class}`~intake_thredds.source.THREDDSMergedSource` accessible via the `.open_thredds_merged()` function. The inputs for this function consists of:

- `url`: top level URL of the THREDDS catalog
- `path`: a list of paths for child catalogs to descend down. The paths can include glob characters (*, ?). These glob characters are used for matching.

In [13]:
source = intake.open_thredds_merged(
    cat_url, path=['Datasets', 'ncep.reanalysis.dailyavgs', 'surface', 'air*sig995*194*.nc']
)
print(source)

sources:
  thredds_merged:
    args:
      path:
      - Datasets
      - ncep.reanalysis.dailyavgs
      - surface
      - air*sig995*194*.nc
      url: https://psl.noaa.gov/thredds/catalog.xml
    description: ''
    driver: intake_thredds.source.THREDDSMergedSource
    metadata: {}



To load the data into an xarray {py:class}`~xarray.Dataset`, you can invoke the `.to_dask()` method. 
Internally, {py:class}`~intake_thredds.source.THREDDSMergedSource` does the following:
- descend down the given paths and collect all available datasets.
- load each dataset in a dataset.
- combine all loaded datasets into a single dataset.

In [14]:
%%time
ds = source.to_dask()
ds

Dataset(s): 100%|████████████████████████████████| 2/2 [00:07<00:00,  3.85s/it]CPU times: user 821 ms, sys: 57.6 ms, total: 879 ms
Wall time: 10.6 s



<xarray.Dataset>
Dimensions:  (lat: 73, lon: 144, time: 731)
Coordinates:
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1948-01-01 1948-01-02 ... 1949-12-31
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(366, 73, 144), meta=np.ndarray>
Attributes:
    Conventions:                     COARDS
    title:                           mean daily NMC reanalysis (1948)
    description:                     Data is from NMC initialized reanalysis\...
    platform:                        Model
    history:                         created 99/05/11 by Hoop (netCDF2.3)\nCo...
    dataset_title:                   NCEP-NCAR Reanalysis 1
    References:                      http://www.psl.noaa.gov/data/gridded/dat...
    DODS_EXTRA.Unlimited_Dimension:  time

## Caching

Under the hood `intake-thredds` uses the `driver='opendap'` from `intake-xarray` by default. You can also choose
`driver='netcdf'`, which in combination with `fsspec` caches files by appending `simplecache::` to the url,
see https://filesystem-spec.readthedocs.io/en/latest/features.html#remote-write-caching.

In [15]:
import os

import fsspec

# specify caching location, where to store files to with their original names
fsspec.config.conf['simplecache'] = {'cache_storage': 'my_caching_folder', 'same_names': True}

cat_url = 'https://psl.noaa.gov/thredds/catalog.xml'
source = intake.open_thredds_merged(
    f'simplecache::{cat_url}',
    path=['Datasets', 'ncep.reanalysis.dailyavgs', 'surface', 'air.sig995.194*.nc'],
    driver='netcdf',  # specify netcdf driver to open HTTPServer
)
print(source)

sources:
  thredds_merged:
    args:
      driver: netcdf
      path:
      - Datasets
      - ncep.reanalysis.dailyavgs
      - surface
      - air.sig995.194*.nc
      url: simplecache::https://psl.noaa.gov/thredds/catalog.xml
    description: ''
    driver: intake_thredds.source.THREDDSMergedSource
    metadata:
      fsspec_pre_url: 'simplecache::'



In [16]:
%time ds = source.to_dask()

Dataset(s): 100%|████████████████████████████████| 2/2 [00:00<00:00, 21.43it/s]CPU times: user 422 ms, sys: 31.4 ms, total: 454 ms
Wall time: 2.89 s



In [17]:
assert os.path.exists('my_caching_folder/air.sig995.1949.nc')

In [18]:
# after caching very fast
%time ds = source.to_dask()

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.2 µs


Another example of this is to read in ensemble members in parallel for GEFS. The example below reads in 21 ensemble members for a single timestep. It also demonstrates usage of `xarray_kwargs` which are passed on to `xarray` for opening the files which in this cases uses the `cfgrib` engine:

In [19]:
cat_url = 'https://www.ncei.noaa.gov/thredds/catalog/model-gefs-003/202008/20200831/catalog.xml'
source = intake.open_thredds_merged(
    f'simplecache::{cat_url}',
    path=["NCEP gens-a Grid 3 Member-Forecast *-372 for 2020-08-31 00:00*"],
    driver="netcdf",
    concat_kwargs={"dim": "number"},
    xarray_kwargs=dict(
        engine="cfgrib",
        backend_kwargs=dict(
            filter_by_keys={"typeOfLevel": "heightAboveGround", "cfVarName": "t2m"}
        ),
    ),
)
source.to_dask()

Dataset(s): 100%|██████████████████████████████| 21/21 [00:00<00:00, 35.56it/s]


<xarray.Dataset>
Dimensions:            (latitude: 181, longitude: 360, number: 21)
Coordinates:
  * number             (number) int64 20 19 18 17 16 15 14 13 ... 6 5 4 3 2 1 0
    time               datetime64[ns] 2020-08-31
    step               timedelta64[ns] 15 days 12:00:00
    heightAboveGround  float64 2.0
  * latitude           (latitude) float64 90.0 89.0 88.0 ... -88.0 -89.0 -90.0
  * longitude          (longitude) float64 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
    valid_time         datetime64[ns] 2020-09-15T12:00:00
Data variables:
    t2m                (number, latitude, longitude) float32 dask.array<chunksize=(1, 181, 360), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          2
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2021-06-09T17:47 GRIB to CDM+CF via cfgrib-0.9.9...